<a href="https://colab.research.google.com/github/robd123/AML/blob/main/Project3/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import urllib.request
import os
datapath = os.path.join("datasets", "heart", "")
os.makedirs(datapath, exist_ok=True)
file_path = "https://raw.githubusercontent.com/robd123/AML/main/Project3/heart.csv"
urllib.request.urlretrieve(file_path, datapath + "heart.csv")

('datasets/heart/heart.csv', <http.client.HTTPMessage at 0x7fbcdab4b410>)

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

heart_data = pd.read_csv(datapath + "heart.csv", thousands=',')
print(heart_data.head())

   age  sex  cp  trestbps  chol  fbs  ...  exang  oldpeak  slope  ca  thal  target
0   63    1   3       145   233    1  ...      0      2.3      0   0     1       1
1   37    1   2       130   250    0  ...      0      3.5      0   0     2       1
2   41    0   1       130   204    0  ...      0      1.4      2   0     2       1
3   56    1   1       120   236    0  ...      0      0.8      2   0     2       1
4   57    0   0       120   354    0  ...      1      0.6      2   0     2       1

[5 rows x 14 columns]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
sns.barplot(x="age",y="target",data=heart_data)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x="sex",y="target",data=heart_data)

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

heart_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

heart_data_scaled = heart_pipeline.fit_transform(heart_data)
heart_data_scaled = pd.DataFrame(heart_data_scaled, columns = heart_data.columns)


In [19]:
heart_data.corr()['target'].sort_values(ascending=False)

AttributeError: ignored